### MobileNet Model Training Emotion Detection

In [31]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import tensorflow as tf
from keras.utils import load_img
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import nbformat

import warnings
warnings.filterwarnings("ignore")

In [32]:
class CFG:
    batch_size = 256
    img_height = 48
    img_width = 48
    epoch = 25

In [33]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [34]:
# Get Number of Images for each Emotion and Splitting

train_path = "Data/train"
test_path = "Data/test"
emotions = os.listdir(train_path)

len_train = {}
len_public_test = {}
len_private_test = {}

for emotion in emotions:
    file_train = os.listdir(os.path.join(train_path, emotion))
    len_train[emotion] = len(file_train)
    
    file_test = os.listdir(os.path.join(test_path, emotion))
    file_public_test = [f for f in file_test if "Public" in f]
    file_private_test = [f for f in file_test if "Private" in f]
    len_public_test[emotion] = len(file_public_test)
    len_private_test[emotion] = len(file_private_test)

print("Train: ", len_train)
print("Public Test: ", len_public_test)
print("Private Test: ", len_private_test)

total_train = sum(len_train.values())
total_public_test = sum(len_public_test.values())
total_private_test = sum(len_private_test.values())

print("Total Train: ", total_train)
print("Total Public Test: ", total_public_test)
print("Total Private Test: ", total_private_test)

Train:  {'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}
Public Test:  {'angry': 467, 'disgust': 56, 'fear': 496, 'happy': 895, 'neutral': 607, 'sad': 653, 'surprise': 415}
Private Test:  {'angry': 491, 'disgust': 55, 'fear': 528, 'happy': 879, 'neutral': 626, 'sad': 594, 'surprise': 416}
Total Train:  28709
Total Public Test:  3589
Total Private Test:  3589


In [35]:
# Class Distribution

# Create Subplots
data_split = ["Train", "Public Test", "Private Test"]
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=data_split
)

all_len = [len_train, len_public_test, len_private_test]
all_total = [total_train, total_public_test, total_private_test]

for i in range(3):
    # Bar Chart
    fig.add_trace(
        go.Bar(
            x=list(all_len[i].keys()),
            y=list(all_len[i].values()),
            name=data_split[i],
            marker_color=["#935d39", "#926f48", "#90825c", "#8e9590", "#8ba0a5", "#88acb9", "#85b8cd"],
        ), row=i+1, col=1
    )

    # Update Axes
    fig.update_xaxes(ticks="outside", linecolor="Black", row=i+1, col=1)
    fig.update_yaxes(ticks="outside", linecolor="Black", row=i+1, col=1)

# Update Layout
fig.update_layout(
    title="Class Distribution", title_x=0.5, font_size=14, font_family="Cambria",
    width=650, height=900,
    plot_bgcolor="White",
    showlegend=False
)

# Show
# fig.show(renderer="iframe_connected")

In [36]:
# Create a new subdirectory called PublicTest and move the public test images into it
if(os.path.exists("public_test")==False):
    os.mkdir("public_test")
for emotion in emotions:
    emotion_dir = "public_test/" + emotion
    if(os.path.exists(emotion_dir)==False):
        os.mkdir(emotion_dir)
    
    first_emotion_dir = os.path.join(test_path, emotion)
    file_test = os.listdir(first_emotion_dir)
    file_public_test = [f for f in file_test if "Public" in f]
    
    for i in range(len(file_public_test)):
        old_path = os.path.join(first_emotion_dir, file_public_test[i])
        new_path = os.path.join(emotion_dir, file_public_test[i])
        shutil.copyfile(old_path, new_path)

# Create a new subdirectory called PrivateTest and move the private test images into it
if(os.path.exists("private_test")==False):
    os.mkdir("private_test")
for emotion in emotions:
    emotion_dir = "private_test/" + emotion
    if(os.path.exists(emotion_dir)==False):
        os.mkdir(emotion_dir)
        
    first_emotion_dir = os.path.join(test_path, emotion)
    file_test = os.listdir(first_emotion_dir)
    file_private_test = [f for f in file_test if "Private" in f]
    
    for i in range(len(file_private_test)):
        old_path = os.path.join(first_emotion_dir, file_private_test[i])
        new_path = os.path.join(emotion_dir, file_private_test[i])
        shutil.copyfile(old_path, new_path)

In [37]:
# Data Augmentation
def data_augmentation():
    # Training Dataset
    TRAINING_DIR = "Data/train"
    train_datagen = ImageDataGenerator(
        rescale=1/255.,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )
    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=CFG.batch_size,
        class_mode="categorical",
        shuffle=True,
        subset="training"
    )

    # Validation Dataset
    validation_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=CFG.batch_size,
        class_mode="categorical",
        shuffle=False,
        subset="validation"
    )

    # Public Testing Dataset
    PUBLIC_TESTING_DIR = "public_test"
    public_test_datagen = ImageDataGenerator(rescale=1./255.)
    public_test_generator = public_test_datagen.flow_from_directory(
        PUBLIC_TESTING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=1,
        class_mode="categorical",
        shuffle=False
    )
    
    # Private Testing Dataset
    PRIVATE_TESTING_DIR = "private_test"
    private_test_datagen = ImageDataGenerator(rescale=1./255.)
    private_test_generator = private_test_datagen.flow_from_directory(
        PRIVATE_TESTING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=1,
        class_mode="categorical",
        shuffle=False
    )
    
    return train_generator, validation_generator, public_test_generator, private_test_generator

In [38]:
# Create data augmentation
seed_everything(2023)
train_generator, validation_generator, public_test_generator, private_test_generator = data_augmentation()

# Load the MobileNet model without the top layer
base_model = MobileNet(
    input_shape=(CFG.img_height, CFG.img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Add a GlobalAveragePooling2D layer and a Dense layer with softmax activation for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Combine the base model with the new layers to create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base model to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-5), metrics=['accuracy'])
model.summary()

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 24, 24, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 24, 24, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 24, 24, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 24, 24, 32)       288       
                      

In [39]:
# Create Callback
model_checkpoint = ModelCheckpoint(
    "MobileNetV1.h5", 
    monitor="val_loss", 
    mode="min", 
    verbose=1,
    save_best_only=True
)

callbacks = [model_checkpoint]

#### Training and testing the data using MobileNet

In [40]:
# Train the model on the training data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // CFG.batch_size,
    epochs=CFG.epoch,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // CFG.batch_size,
    callbacks=callbacks,
)

Epoch 1/25
89/89 [==============================] - ETA: 0s - loss: 1.9654 - accuracy: 0.1279
Epoch 1: val_loss improved from inf to 1.94035, saving model to MobileNetV1.h5
89/89 [==============================] - 33s 353ms/step - loss: 1.9654 - accuracy: 0.1279 - val_loss: 1.9403 - val_accuracy: 0.1827
Epoch 2/25
89/89 [==============================] - ETA: 0s - loss: 1.9448 - accuracy: 0.2059
Epoch 2: val_loss improved from 1.94035 to 1.92307, saving model to MobileNetV1.h5
89/89 [==============================] - 33s 366ms/step - loss: 1.9448 - accuracy: 0.2059 - val_loss: 1.9231 - val_accuracy: 0.2528
Epoch 3/25
89/89 [==============================] - ETA: 0s - loss: 1.9289 - accuracy: 0.2246
Epoch 3: val_loss improved from 1.92307 to 1.90740, saving model to MobileNetV1.h5
89/89 [==============================] - 35s 388ms/step - loss: 1.9289 - accuracy: 0.2246 - val_loss: 1.9074 - val_accuracy: 0.2578
Epoch 4/25
89/89 [==============================] - ETA: 0s - loss: 1.9161 - 

#### MobileNet Results

In [41]:
# Evaluate on Public Test Data
public_scores = model.evaluate(public_test_generator)
print("%s: %.2f%%" % ("Evaluate Public Test Accuracy", public_scores[1]*100))

# Evaluate on Private Test Data
private_scores = model.evaluate(private_test_generator)
print("%s: %.2f%%" % ("Evaluate Private Test Accuracy", private_scores[1]*100))

3589/3589 [==============================] - 18s 5ms/step - loss: 1.8089 - accuracy: 0.2510
Evaluate Public Test Accuracy: 25.10%
3589/3589 [==============================] - 15s 4ms/step - loss: 1.8148 - accuracy: 0.2458
Evaluate Private Test Accuracy: 24.58%


In [42]:
# Visualize Training and Validation Results

# Create Subplot
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Model Loss", "Model Accuracy"], 
)

# Loss Plot
loss = history.history['loss']
val_loss = history.history['val_loss']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(loss)+1), y=loss,
        mode="markers+lines",
        marker=dict(
            color="#935d39", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#935d39", width=1.5),
        name="Training Loss"
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_loss)+1), y=val_loss,
        mode="markers+lines",
        marker=dict(
            color="#85b8cd", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#85b8cd", width=1.5),
        name="Validation Loss"
    ), row=1, col=1
)

# Accuracy Plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(acc)+1), y=acc,
        mode="markers+lines",
        marker=dict(
            color="#935d39", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#935d39", width=1.5),
        name="Training Accuracy"
    ), row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_acc)+1), y=val_acc,
        mode="markers+lines",
        marker=dict(
            color="#85b8cd", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#85b8cd", width=1.5),
        name="Validation Accuracy"
    ), row=1, col=2
)

# Update Axes
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_yaxes(title="Categorical Loss", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_yaxes(title="Accuracy", linecolor="Black", ticks="outside", row=1, col=2)

# Update Layout
fig.update_layout(
    title="Training and Validation Results", title_x=0.5, font_family="Cambria",
    width=950, height=400,
    showlegend=False,
    plot_bgcolor="White",
    paper_bgcolor="White"
)

# Show
fig.show(iframe_connected=True)

In [43]:
# Confusion Matrix
predictions = model.predict(public_test_generator)

# Get the true labels from the generator
true_labels = public_test_generator.classes

# Compute the confusion matrix using tf.math.confusion_matrix
confusion_matrix = tf.math.confusion_matrix(
        labels=true_labels,
        predictions=predictions.argmax(axis=1),
        num_classes=7)

# Print the confusion matrix
print(confusion_matrix)

3589/3589 [==============================] - 11s 3ms/step
tf.Tensor(
[[  0   0   0 465   2   0   0]
 [  0   0   0  56   0   0   0]
 [  0   0   0 494   2   0   0]
 [  0   0   0 895   0   0   0]
 [  0   0   0 604   3   0   0]
 [  0   0   0 652   1   0   0]
 [  0   0   0 410   2   0   3]], shape=(7, 7), dtype=int32)
